# Autoencoder

In [1]:
import os

from utils.loaders import load_mnist
from models.AE import Autoencoder

Using TensorFlow backend.


## Set parameters

In [3]:
# run params
SECTION = 'vae'
RUN_ID = '0001'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

MODE =  'build' #'load' #

## Load the data

In [4]:
(x_train, y_train), (x_test, y_test) = load_mnist()

## Define the structure of the autoencoder (neural network)

Generally speaking, it is a good idea to create a class for our model in a separate file.

* This way, we can instantiate an `Autoencoder` object with parameters that define a particular model architecture in the notebook, as shown below. 

* This makes our model very flexible and able to be easily tested and ported to other projects as necessary.


In [5]:
# If code fails, run: conda install -c anaconda graphviz

# Define the structure of the autoencoder
AE = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if MODE == 'build':
    AE.save(RUN_FOLDER)
else:
    AE.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

W0116 16:26:19.716855 13752 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0116 16:26:19.759363 13752 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0116 16:26:19.788359 13752 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Let us now take a look at the architecture of an autoencoder in more detail, starting with the encoder.

## The Encoder

In an autoencoder, the encoder’s job is to take the input image and map it to a point in the **latent space**. 

The architecture of the encoder we will be building is shown in **Figure 5**.

In [6]:
AE.encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 28, 28, 1)         0         
_________________________________________________________________
encoder_conv_0 (Conv2D)      (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 28, 28, 32)        0         
_________________________________________________________________
encoder_conv_1 (Conv2D)      (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
encoder_conv_2 (Conv2D)      (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 7, 7, 64)          0         
__________

**Figure 5**: Architecture of the encoder.

To achieve this, we first create an input layer for the image and pass this through four `Conv2D` (see: encoder_conv_0 to encoder_conv_3) layers in sequence, each capturing increasingly high-level features. 

* We use a stride of 2 on some of the layers to reduce the size of the output. 
* The last convolutional layer is then flattened and connected to a `Dense` layer of size 2, which represents our two-dimensional **latent space**.

**Example 2** below shows how to build this in Keras.

---
**Example #2**: The encoder
<img src="./images/example_2.png" width="700" border="1">


* We can change the number of convolutional layers in the encoder simply by adding elements to the lists that define the model architecture in the notebook. 


* It is strongly recommend experimenting with the parameters that define the models, to understand how the architecture affects the number of weights in each layer, model performance, and model runtime.

## The Decoder

The decoder is a mirror image of the encoder, except instead of convolutional layers, we use *`convolutional transpose`* layers, as shown in **Figure 6**.

In [7]:
AE.decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 64)        0         
__________

**Figure 6**: Architecture of the decoder.

**Note** that the decoder doesn’t have to be a mirror image of the encoder. 
* It can be anything you want, as long as the output from the last layer of the decoder is the same size as the input to the encoder (why?: **since our loss function will be comparing these pixelwise**).

## Convolutional Transpose Layers

* Standard convolutional layers allow us to halve the size of an input tensor in both height and width, by setting strides = 2.


* The convolutional transpose layer uses the same principle as a standard convolutional layer (passing a filter across the image), but is different in that setting strides = 2 doubles the size of the input tensor in both height and width.


* In a convolutional transpose layer, the strides parameter determines the internal zero padding between pixels in the image as shown in **Figure 7**.

<img src="./images/figure_7.png" width="700" border="1">

**Figure 7**: A convolutional transpose layer example—here, a 3 × 3 × 1 filter (gray) is being passed across a 3 × 3 × 1 image (blue) with strides = 2, to produce a 6 × 6 × 1 output tensor (green).

`Source: Vincent Dumoulin and Francesco Visin, “A Guide to Convolution Arithmetic for Deep Learning,” 12
January 2018, https://arxiv.org/pdf/1603.07285.pdf.`

In Keras, the `Conv2DTranspose` layer allows us to perform convolutional transpose operations on tensors. 

* By stacking these layers, we can gradually expand the size of each layer, using strides of 2, until we get back to the original image dimension of 28 × 28.


**Example 3** shows how we build the decoder in Keras.

---
**Example 3**: The decoder
<img src="./images/example_3.png" width="700" border="1">

## Joining the Encoder to the Decoder
To train the encoder and decoder simultaneously, we need to define a model that will represent the flow of an image through the encoder and back out through the decoder. 

Luckily, Keras makes it extremely easy to do this, as we can see in **Example 4**.

---
**Example 4**: The full autoencoder
<img src="./images/example_4.png" width="700" border="1">

Now that we have defined our model, we just need to **compile** it with a **loss function** and **optimizer**, as shown in **Example 5**. 

* The **loss function** is usually chosen to be either 
    * the root mean squared error (RMSE) or 
    * binary cross-entropy between the individual pixels of the original image and the reconstruction. 



* Binary cross-entropy places heavier penalties on predictions at the extremes that are badly wrong, so it tends to push pixel predictions to the middle of the range. 
    * This results in less vibrant images. 
    


* For this reason, it is generally preferred to use RMSE as the loss function. 

However, there is no right or wrong choice—we should choose whichever works best for our use case.

---
**Example 5**: Compilation
<img src="./images/example_5.png" width="450" border="1">

We can now train the autoencoder by passing in the input images as both the input and output, as shown in **Example 6**.

---
**Example 6**: Training the autoencoder
<img src="./images/example_6.png" width="280" border="1">

## Train the autoencoder

In [8]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0

In [9]:
AE.compile(LEARNING_RATE)

W0116 16:26:43.538507 13752 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [10]:
AE.train(     
    x_train[:1000]
    , batch_size = BATCH_SIZE
    , epochs = 200
    , run_folder = RUN_FOLDER
    , initial_epoch = INITIAL_EPOCH
)

W0116 16:26:48.123074 13752 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0116 16:26:48.410201 13752 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/200
1000/1000 [==============================] - 4s 4ms/step - loss: 0.1677

Epoch 00001: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 2/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0765

Epoch 00002: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 3/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0679

Epoch 00003: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 4/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0665

Epoch 00004: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 5/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0650

Epoch 00005: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 6/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0612

Epoch 00006: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 7/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.059

1000/1000 [==============================] - 3s 3ms/step - loss: 0.0441

Epoch 00054: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 55/200
1000/1000 [==============================] - 4s 4ms/step - loss: 0.0439

Epoch 00055: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 56/200
1000/1000 [==============================] - 4s 4ms/step - loss: 0.0435

Epoch 00056: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 57/200
1000/1000 [==============================] - 4s 4ms/step - loss: 0.0435

Epoch 00057: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 58/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0436

Epoch 00058: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 59/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0434

Epoch 00059: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 60/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0433

Epo

1000/1000 [==============================] - 3s 3ms/step - loss: 0.0365

Epoch 00160: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 161/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0364

Epoch 00161: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 162/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0362

Epoch 00162: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 163/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0362

Epoch 00163: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 164/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0364

Epoch 00164: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 165/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0366

Epoch 00165: saving model to run/vae/0001_digits\weights/weights.h5
Epoch 166/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.036

END.